In [6]:
import h2o4gpu
import h2o4gpu.util.import_data as io
import h2o4gpu.util.metrics as metrics
from tabulate import tabulate
from h2o4gpu.solvers.linear_regression import LinearRegression
import pandas as pd
import os, sys

In [7]:
"""
Import Data for H2O GPU Edition

This function will read in data and prepare it for H2O4GPU's GLM solver

Parameters
----------
data_path : str
             A path to a dataset (The dataset needs to be all numeric)
use_pandas : bool
              Indicate if Pandas should be used to parse
intercept : bool
              Indicate if intercept term is needed
valid_fraction : float
                  Percentage of dataset reserved for a validation set
classification : bool
                  Classification problem?
Returns
-------
If valid_fraction > 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    valid_x: numpy array of valid input variables
    valid_y: numpy array of valid y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
If valid_fraction == 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
"""
if not os.path.exists("ipums_1k.csv"):
    !wget https://s3.amazonaws.com/h2o-public-test-data/h2o4gpu/open_data/ipums_1k.csv
train_x,train_y,valid_x,valid_y,family=io.import_data(data_path="ipums_1k.csv", 
                                                        use_pandas=True, 
                                                        intercept=True,
                                                        valid_fraction=0.2,
                                                        classification=False)

Reading Data with Pandas
(999, 9733)
Original m=999 n=9732
Size of Train rows=800 & valid rows=199
Size of Train cols=9732 valid cols=9732
Size of Train cols=9733 & valid cols=9733 after adding intercept column


In [8]:
"""
Set up instance of H2O4GPU's GLM solver with default parameters

Need to pass in `family` to indicate problem type to solve
"""
print("Setting up solver")
model = LinearRegression()

Setting up solver


In [9]:
"""
Fit Linear Regression Solver
"""
print("Solving")
%time model.fit(train_x, train_y)
print("Done Solving")
model.summary()

Solving
CPU times: user 7.04 s, sys: 1.4 s, total: 8.43 s
Wall time: 8.5 s
Done Solving
RMSE per alpha value (-1.00 = missing)

|   Alphas |   Train |    CV |   Valid |
|---------:|--------:|------:|--------:|
|     0.00 | 4321.38 | -1.00 |   -1.00 |


In [10]:
"""
Make predictions on validation set
"""
print("Predictions per alpha")
preds = model.predict(valid_x, valid_y)
print(preds)

Predictions per alpha
Detected np.float32 data
Detected np.float32 data
Detected np.float32
single precision fit
thecount=199 count_full_value=0 count_short_value=9740 n=9734 num_all_other=6 m_valid=199
[[  1.11381211e+04   8.42791094e+04   4.05828086e+04   5.35622539e+04
    4.26661406e+04   2.63907891e+04   1.66206738e+04   4.01061250e+04
    4.61646562e+04   5.76909180e+04  -1.49988643e+04   1.43937783e+04
    2.21791621e+04   2.49092227e+04   8.64333594e+04   5.05008594e+04
    3.24703652e+04   2.15377168e+04   1.15320312e+04   1.02565742e+05
   -5.53202454e+02   2.43787344e+04   9.61504102e+03   5.70016504e+03
    1.52409033e+04   6.35301523e+04   6.69761172e+04   7.89945312e+02
    1.85213574e+04  -2.26827070e+04   4.28969453e+04   3.92351680e+04
   -4.89953789e+04   1.53601260e+04  -3.53628613e+03  -5.82679883e+03
    3.02814473e+04   1.09081777e+04  -6.70460596e+03   1.27039984e+05
   -3.67764062e+04  -3.43336250e+04   8.08519844e+04   2.71479863e+04
   -3.28139805e+04   1.7025

In [11]:
"""
Get logloss or rmse for validation set per alpha
"""
for i in range(model.n_alphas):
    if family == "logistic":
        print("Logloss for alpha = ",model.alphas_best[i])
        print(metrics.log_loss(valid_y, preds[i]))
    else:
        print("RMSE for alpha = ",model.alphas_best[i])
        print(metrics.rmse(valid_y,preds[i]))

RMSE for alpha =  [ 0.]
45457.8
